In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
from tqdm.notebook import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 

In [ ]:
# read data
data_path = "../input/cassava-leaf-disease-classification"
train_img_path = os.path.join(data_path, "train_images")
train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
train_df.head()

In [ ]:
# EDA
print("Total Number of Train： {}".format(len(train_df)))
print()
print("Each Ratio of Train data:")
print(train_df['label'].value_counts()/len(train_df))
(train_df['label'].value_counts()/len(train_df)).plot(kind='bar')
plt.show()

In [ ]:
def plot_images(df, data_img_path, figsize=(30,20)):
    fig = plt.figure(figsize=figsize)
    len_train = len(train_df["label"].unique())
    for i in range(len_train):
        ax = fig.add_subplot(len_train, 1, i+1)
        sample = df[df['label']==i]['image_id'].sample().reset_index(drop=True)
        sample_img_path = os.path.join(data_img_path, sample.iloc[0])
        img = cv2.imread(sample_img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title("Label: {}, Shape: {}".format(i, img.shape))
    
    plt.show()
    
plot_images(train_df, train_img_path)

In [ ]:
# create Dataset and Transform
class CassavaDataset(Dataset):
    def __init__(self, df:pd.DataFrame, data_img_path:str, phase:str='train', transform=None):
        self.df = df
        self.data_img_path = data_img_path
        self.transform = transform
        self.phase = phase
        
    def __getitem__(self, idx):
        sample = self.df['image_id'].iloc[idx]
        img = cv2.imread(os.path.join(self.data_img_path, sample))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            img = self.transform(img, self.phase)
            
        if self.phase=='test':
            return img
        
        label = self.df['label'].iloc[idx]
        
        return img, label
    
    def __len__(self):
        return len(self.df)
    
class Transform(object):
    def __init__(self, resize:int, mean:list, std:list, crop_size:int):
        self.transform = {
            'train':A.Compose([
                A.Resize(resize, resize),
                A.RandomCrop(height=crop_size, width=crop_size),
                A.Normalize(mean, std),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                ToTensorV2()
            ]),
            'val':A.Compose([
                A.Resize(resize, resize),
                A.CenterCrop(crop_size, crop_size),
                A.Normalize(mean, std),
                ToTensorV2()
            ]),
            'test':A.Compose([
                A.Resize(resize, resize),
                A.CenterCrop(crop_size, crop_size),
                A.Normalize(mean, std),
                ToTensorV2()
            ]),
        }
    
    def __call__(self, img, phase):
        return self.transform[phase](image=img)['image']
    
resize = 256
crop_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = Transform(resize, mean, std, crop_size)

# test dataset and transform
dataset = CassavaDataset(train_df, train_img_path, 'train', transform)
dataloader = DataLoader(dataset, shuffle=True, batch_size=1, num_workers=2)
iterator = dataloader.__iter__()
img, label = next(iterator)
plt.imshow(img.numpy().transpose(2,3,1,0).reshape(crop_size, crop_size, 3))
plt.title("Dataset and Transform Test, Label: {}".format(label))
plt.axis('off')
plt.show()

In [ ]:
# create model
def conv3x3(in_channels, out_channels, stride=1, groups=1):
    return nn.Conv2d(
        in_channels, out_channels, kernel_size=3, stride=stride, padding=1, groups=groups, bias=False,
    )


def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels, out_channels, kernel_size=1, stride=stride, bias=False
    )


class Bottleneck(nn.Module):
    expansion = 4  # 出力のチャンネル数を入力のチャンネル数の何倍に拡大するか

    def __init__(self, in_channels, channels, stride=1, cardinality=1, base_width=64):
        super().__init__()
        width = int(channels * (base_width / 64)) * cardinality
        self.conv1 = conv1x1(in_channels, width)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = conv3x3(width, width, stride, groups=cardinality)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = conv1x1(width, channels * self.expansion)
        self.bn3 = nn.BatchNorm2d(channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合、x をダウンサンプリングする。
        if in_channels != channels * self.expansion:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(x)

        out = self.relu(out)

        return out

class ResNeXt(nn.Module):
    def __init__(self, layers, num_classes=1000, cardinality=1, base_width=64):
        super().__init__()

        self.in_channels = 64
        self.cardinality = cardinality
        self.base_width = base_width
        self.conv1 = nn.Conv2d(
            3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, layers[0])
        self.layer2 = self._make_layer(128, layers[1], stride=2)
        self.layer3 = self._make_layer(256, layers[2], stride=2)
        self.layer4 = self._make_layer(512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * Bottleneck.expansion, num_classes)

        # 重みを初期化する。
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, channels, blocks, stride=1):
        layers = []

        layers.append(
            Bottleneck(
                self.in_channels,
                channels,
                stride,
                cardinality=self.cardinality,
                base_width=self.base_width,
            )
        )

        self.in_channels = channels * Bottleneck.expansion
        for _ in range(1, blocks):
            layers.append(
                Bottleneck(
                    self.in_channels,
                    channels,
                    cardinality=self.cardinality,
                    base_width=self.base_width,
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
    
def resnext50_32x4d():
    return ResNeXt([3, 4, 6, 3], cardinality=32, base_width=4)

model = resnext50_32x4d()

In [1]:
def train_model(model, dt_dataloader, criterion, optimizer, epochs, device='cpu', sheduler=None):
    since = time.time()
    best_acc = 0.0
    
    if device == 'cuda':
        model = nn.DataParallel(model)
        torch.backends.cudnn.benchmark = True
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        print('-'*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dt_dataloader[phase]):
                model = model.to(device)
                inputs = inputs.to(device)
                labels = labels.to(device) 
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==labels)
                
            epoch_loss = running_loss / len(dt_dataloader[phase].dataset)
            epoch_acc = running_corrects / len(dt_dataloader[phase].dataset)
            
            if phase == 'val' and best_acc < epoch_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'best_resnext.pth')
        
            if phase == 'train' and scheduler != None:
                scheduler.step()
            
            print('{} Loss: {:4f}, Acc: {:4f}'.format(phase, epoch_loss, epoch_acc))
        print()
    
    elapced_time = time.time() - since
    print("Total train time: {}m {}s".format(elapced_time//60, elapced_time%60))
    print("Val best acc: {}".format(best_acc))
    
train, val = train_test_split(train_df, test_size=0.2, random_state=42)
traindataset = CassavaDataset(train, train_img_path, 'train', transform)
valdataset = CassavaDataset(val, train_img_path, 'val', transform)
traindl = DataLoader(traindataset, batch_size=16, shuffle=True, num_workers=2)
valdl = DataLoader(valdataset, batch_size=16, shuffle=True, num_workers=2)

dl_dict = {'train':traindl, 'val':valdl}
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.5)
epochs = 5
train_model(model, dl_dict, criterion, optimizer, epochs, device, scheduler)

NameError: name 'train_test_split' is not defined

In [ ]:
# test
model.load_state_dict(torch.load(os.path.join(data_path, 'baseline-resnext/best_resnext.pth')))
model.eval()
model = model.to(device)

test_df = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
testdataset = CassavaDataset(test_df, os.path.join(data_path, 'test_images'), 'test', transform)
testdl = DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=2)

predictions = []
for inputs in testdl:
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, preds = torch.max(preds, 1)
    predictions.append(preds)
    
test_df['label'] = np.concatenate(predictions)
test_df.to_csv('submission.csv')